In [1]:
import pandas as pd

### Loading the data

In [2]:
ls

Final_notebook.ipynb    README.md               Working_notebook.ipynb


In [3]:
df = pd.read_csv('/Users/raylinsoriano/Documents/Flatiron/capstone/data/listings (1).csv')

In [4]:
df

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20210901220703,2021-09-02,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/f0813a11-40b2...,2845,https://www.airbnb.com/users/show/2845,...,4.79,4.86,4.41,NaN,f,3,3,0,0,0.33
1,3831,https://www.airbnb.com/rooms/3831,20210901220703,2021-09-02,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",Just the right mix of urban center and local n...,https://a0.muscache.com/pictures/e49999c2-9fd5...,4869,https://www.airbnb.com/users/show/4869,...,4.81,4.72,4.65,NaN,f,1,1,0,0,5.03
2,5121,https://www.airbnb.com/rooms/5121,20210901220703,2021-09-02,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,NaN,https://a0.muscache.com/pictures/2090980c-b68e...,7356,https://www.airbnb.com/users/show/7356,...,4.91,4.47,4.52,NaN,f,1,0,1,0,0.54
3,5136,https://www.airbnb.com/rooms/5136,20210901220703,2021-09-02,"Spacious Brooklyn Duplex, Patio + Garden",We welcome you to stay in our lovely 2 br dupl...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,7378,https://www.airbnb.com/users/show/7378,...,5.00,4.50,5.00,NaN,f,1,1,0,0,0.02
4,5178,https://www.airbnb.com/rooms/5178,20210901220703,2021-09-02,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"Theater district, many restaurants around here.",https://a0.muscache.com/pictures/12065/f070997...,8967,https://www.airbnb.com/users/show/8967,...,4.40,4.86,4.36,NaN,f,1,0,1,0,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36918,52002317,https://www.airbnb.com/rooms/52002317,20210901220703,2021-09-02,HUGE ROOM IN IDEAL LES SPOT,Extremely Huge Room in the IDEAL Lower East Si...,NaN,https://a0.muscache.com/pictures/40bc2a16-7609...,283057520,https://www.airbnb.com/users/show/283057520,...,NaN,NaN,NaN,NaN,t,4,0,4,0,NaN
36919,52004427,https://www.airbnb.com/rooms/52004427,20210901220703,2021-09-02,Furnished Cozy Bedroom in East Village Location,"Full room in a 4 bedroom, 2 bathroom shared ho...",NaN,https://a0.muscache.com/pictures/361bd18b-3f23...,305240193,https://www.airbnb.com/users/show/305240193,...,NaN,NaN,NaN,NaN,f,307,56,251,0,NaN
36920,52005507,https://www.airbnb.com/rooms/52005507,20210901220703,2021-09-03,Bedroom Near Train w/ Washer Dryer,Bedroom Near Train w/ Washer Dryer,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,9372363,https://www.airbnb.com/users/show/9372363,...,NaN,NaN,NaN,NaN,f,3,0,3,0,NaN
36921,52005691,https://www.airbnb.com/rooms/52005691,20210901220703,2021-09-02,Furnished Cozy Bedroom in East Village Location,"Full room in a 3 bedroom, 1 bathroom shared ho...",NaN,https://a0.muscache.com/pictures/958369fa-05c1...,305240193,https://www.airbnb.com/users/show/305240193,...,NaN,NaN,NaN,NaN,f,307,56,251,0,NaN
